In [2]:
import random as rnd
Integer = rnd.randint(1,1024)

print(f"{Integer} as byte string = {Integer:b}") # b as in ~byte string~
print(f"{Integer} as hexadecimal string = {hex(Integer)}") # hex as in ~hexadecimal string~

595 as byte string = 1001010011
595 as hexadecimal string = 0x253


### Briefly about UTF (Unicode Transformation Format)
- UTF-8 encodes characters into binary strings of one, two, three, or four bytes.
- UTF-16 encodes a character into a string of either two or four bytes.

As of Python 3, [all strings are unicode.](https://docs.python.org/3/howto/unicode.html)


In [3]:
str="I'm a string and I must be encoded before hashing."

# convert string to bytes
encoded=str.encode(encoding="UTF-8") # UTF-8 is default (using one to four one-byte (8-bit) code units)

encoded

b"I'm a string and I must be encoded before hashing."

In [4]:
encoded.decode(encoding = "UTF-8")

"I'm a string and I must be encoded before hashing."

In [5]:
foo = str.encode(encoding = "UTF-16") # Encoded UTF-16
print(foo)

b"\xff\xfeI\x00'\x00m\x00 \x00a\x00 \x00s\x00t\x00r\x00i\x00n\x00g\x00 \x00a\x00n\x00d\x00 \x00I\x00 \x00m\x00u\x00s\x00t\x00 \x00b\x00e\x00 \x00e\x00n\x00c\x00o\x00d\x00e\x00d\x00 \x00b\x00e\x00f\x00o\x00r\x00e\x00 \x00h\x00a\x00s\x00h\x00i\x00n\x00g\x00.\x00"


In [6]:
foo.decode(encoding = "UTF-16") # Decoded

"I'm a string and I must be encoded before hashing."

As shown, encoding and decoding is easily done with Python. UTF-16 is not automatically decoded, while UTF-8 is, as the default encoding for Python is UTF-8. (Probably ?)

---
## Simple hashing with SHA-256

In [7]:
import hashlib

# Using SHA-256
hashed = hashlib.sha256(foo)

hashed.hexdigest() # Hash value as hex string (binary value of the string in hexadecimal notation)

'2f8ce08c2cc72327b030624aad0acf3c2dfcc0d4ad816018c6f5c1bdf64b97b5'

In [8]:
print(f"{hashed.digest_size=}\n{hashed.block_size=}") # Show digest and block size

hashed.digest_size=32
hashed.block_size=64


In [9]:
print(hashed.digest()) # Hash value as bytes object (Output)

b"/\x8c\xe0\x8c,\xc7#'\xb00bJ\xad\n\xcf<-\xfc\xc0\xd4\xad\x81`\x18\xc6\xf5\xc1\xbd\xf6K\x97\xb5"


This ""cannot"" be decrypted. However, we can check if input is matching. For example:

In [10]:
my_password = "password".encode()

hashed_password = hashlib.sha256(my_password)

print(hashed_password.digest())

b"^\x88H\x98\xda(\x04qQ\xd0\xe5o\x8d\xc6)'s`=\rj\xab\xbd\xd6*\x11\xefr\x1d\x15B\xd8"


In [11]:
user_input1 = "wordpass".encode()
user_input2 = "password".encode()

login1 = hashlib.sha256(user_input1)
login2 = hashlib.sha256(user_input2)

print(hashed_password.digest() == login1.digest())
print(hashed_password.digest() == login2.digest())

False
True


---

## Login/authentication model with hashlib

It's a mess, but it works??

Just add some error handling and store user keys in better format?

In [12]:
from __future__ import annotations

# Probably shouldn't save user credentials in running Python dictionary, but uhhh, yea.

# Store key pair for user id and hashed password, hex value
super_safe_dictionary = {
    
    "user0": 'f4c628bd9e27adddfda2f883edd88f7d8aa74e713d7e7e6fdbbd581a12b084a5',
    "user1": '89501b04efa10ea485d2c338c6685155f7a0cc4e437d9eb604ccc81b8bf1b178'    
    
    }


def utf16(i):
    """
    Function to make sure all strings have valid and compatible unicode.
    Not sure if this is neccessary at all, or if it makes any different at the moment.
    """
    always_string = i.__str__()
    return always_string.encode(encoding = "UTF-16")


class pin:
    """
    Creates unique hashed passwords.
    no idea if it's even remotely safe
    """
    
    def __init__(self, uid: str, pw: str) -> None:
        self._uid = uid
        self._pw = pw

    def get_uid(self) -> str:
        return self._uid

    def set_uid(self, value) -> None:
        self._uid = value

    def get_pw(self) -> str:
        "If called before hash_pw(), return plaintext pw to check for requirements, etc."

        return self._pw
    
    def hash_pw(self, value: str) -> None:
        "Creates hash from user id and password"
        # TODO: exception handling

        print("hash_pw running")

        # Encode user id and password
        id_encoded = utf16(self._uid)
        value = utf16(value)

        # Take last character from user id for flavor
        soy_sauce = self._uid[-1]
        soy_sauce = utf16(soy_sauce)

        # Merge UTF-16 encoded values of password and user id to create unique hash value
        super_safe_password = soy_sauce + id_encoded + value + soy_sauce
        hash = hashlib.sha256(super_safe_password)
        self._pw = hash.digest()


# Testing new user creation

# Enter user id and password testing
user_input_id = "user2"
user_input_pw = "user2_password"

# If new user:
new_user_instance = pin(user_input_id,user_input_pw)

# Plaintext password, only here for testing
print(f"{new_user_instance.get_pw()=}")

# Print user id, just for testing
print(f"{new_user_instance.get_uid()=}")

# Hash user password
new_user_instance.hash_pw(user_input_pw)

# Print hash value, just for testing
print(f"pw hash value as {type(new_user_instance.get_pw())}: {new_user_instance.get_pw()}")

#Convert to hex values and save in dictionary - probably should convert it to hex in with class method instead
a = new_user_instance.get_pw()
a = a.hex()

print(f"As hex: {a}")

super_safe_dictionary[new_user_instance.get_uid()] = a

super_safe_dictionary



new_user_instance.get_pw()='user2_password'
new_user_instance.get_uid()='user2'
hash_pw running
pw hash value as <class 'bytes'>: b'\xca\x8e\xdf\xfe"\xc5,/\xe4\xeb\xfc\x08">\xbe\x03\xa2\x7f_o\xf2\n\xbdo\xac\x11\x06\xd2BC\x0e\xce'
As hex: ca8edffe22c52c2fe4ebfc08223ebe03a27f5f6ff20abd6fac1106d242430ece


{'user0': 'f4c628bd9e27adddfda2f883edd88f7d8aa74e713d7e7e6fdbbd581a12b084a5',
 'user1': '89501b04efa10ea485d2c338c6685155f7a0cc4e437d9eb604ccc81b8bf1b178',
 'user2': 'ca8edffe22c52c2fe4ebfc08223ebe03a27f5f6ff20abd6fac1106d242430ece'}

Nice. We can create key value pairs. Let's check if we can log in.

In [37]:
my_user_id = "user2"

if my_user_id in super_safe_dictionary:
    print(f"{my_user_id} found. Enter password:")
    my_password = "user2_password"

    print(f"Entered password: {my_password}")

    login_session = pin(my_user_id, my_password)

    login_session.hash_pw(my_password)
    login_session.set_uid(my_user_id)

    # and yeah we gotta do the hex conversion again
    a = login_session.get_pw()
    a = a.hex()
    
    if a == super_safe_dictionary.get(my_user_id):
        print("Success")
    else:
        print("Incorrect password")

else:
    print(f"{my_user_id} does not exist. Create user?")

user2 found. Enter password:
Entered password: user2_password
hash_pw running
Success


Nice.